# Project Title
### Data Engineering Capstone Project

#### Project Summary
--describe your project at a high level--

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [241]:
# Do all imports and installs here
import logging
import numpy as np
import pandas as pd
import psycopg2
from pprint import pprint
import configparser
import psycopg2
import pandas as pd
pd.set_option('display.max_columns', None)
import os
from pyspark.sql import SparkSession, DataFrame
import pyspark.sql.functions as F 
from pyspark.sql.types import StructType as R, StructField as Fld, DoubleType as Dbl, StringType as Str, IntegerType as Int, DateType as Date
from collections import OrderedDict 
from datetime import datetime, timedelta

logging.basicConfig(level=logging.INFO, format='%(asctime)s %(levelname)s  [%(name)s] %(message)s')
LOG = logging.getLogger('etl')

### Step 1: Scope the Project and Gather Data

#### Scope 
Explain what you plan to do in the project in more detail. What data do you use? What is your end solution look like? What tools did you use? etc>

#### Describe and Gather Data 
Describe the data sets you're using. Where did it come from? What type of information is included? 

In [2]:
# Read in the data here
imm_sample = pd.read_csv('./immigration_data_sample.csv')
imm_sample.head()
imm_header = list(imm_sample.head(0))
imm_header = imm_header[1:]
pprint(imm_header)

['cicid',
 'i94yr',
 'i94mon',
 'i94cit',
 'i94res',
 'i94port',
 'arrdate',
 'i94mode',
 'i94addr',
 'depdate',
 'i94bir',
 'i94visa',
 'count',
 'dtadfile',
 'visapost',
 'occup',
 'entdepa',
 'entdepd',
 'entdepu',
 'matflag',
 'biryear',
 'dtaddto',
 'gender',
 'insnum',
 'airline',
 'admnum',
 'fltno',
 'visatype']


In [114]:
# .config("spark.jars.packages","org.apache.hadoop:hadoop-aws:2.7.0") \
# .config("spark.jars.packages", "saurfang:spark-sas7bdat:3.0.0-s_2.12") \
from pyspark.sql import SparkSession
spark = SparkSession.builder \
                     .config("spark.jars.packages","org.apache.hadoop:hadoop-aws:2.7.0") \
                    .enableHiveSupport() \
                    .getOrCreate()
# df_spark =spark.read.format('com.github.saurfang.sas.spark').load('../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat')

In [4]:
#write to parquet
# df_spark.write.parquet("sas_data")
# df_spark=spark.read.parquet("sas_data")

### Step 2: Explore and Assess the Data
#### Explore the Data 
Identify data quality issues, like missing values, duplicate data, etc.

#### Cleaning Steps
Document steps necessary to clean the data

# Performing cleaning tasks here
* Checkout i94 2016 data
* Checkout demographics data
* Checkout airport data
* Checkout temperature data

# get valid values of I94 fields

In [422]:
# try configparsing
config = configparser.ConfigParser()
config.read_file(open('meta_i94.cfg'))

pprint(config['I94MODE'])

<Section: I94MODE>


In [423]:
sections_dict = {}
# get sections and iterate over each
sections = config.sections()

for section in sections:
    options = config.options(section)
    temp_dict = {}
    for option in options:
        k = option.strip('\'').strip().upper()
        v = config.get(section,option).strip('\'').strip().upper()
        temp_dict[k] = v

    sections_dict[section] = temp_dict

In [428]:
# cast meta key into it's original type
meta_i94 = {}
for k, v in sections_dict.items():
    print(k)
    if k == 'I94CIT' or k == 'I94MODE' or k == 'I94VISA':
        meta_i94[k] = {int(vk):vv for vk,vv in v.items()}
    else:
        meta_i94[k] = v

I94CIT
I94PORT
I94MODE
I94VISA
I94ADDR


## i94 data

In [168]:
i94_fnames=['./data/18-83510-I94-Data-2016/i94_jan16_sub.sas7bdat',
            './data/18-83510-I94-Data-2016/i94_feb16_sub.sas7bdat',
            './data/18-83510-I94-Data-2016/i94_mar16_sub.sas7bdat',
            './data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat',
            './data/18-83510-I94-Data-2016/i94_may16_sub.sas7bdat',
            './data/18-83510-I94-Data-2016/i94_jun16_sub.sas7bdat',
            './data/18-83510-I94-Data-2016/i94_jul16_sub.sas7bdat',
            './data/18-83510-I94-Data-2016/i94_aug16_sub.sas7bdat',
            './data/18-83510-I94-Data-2016/i94_sep16_sub.sas7bdat',
            './data/18-83510-I94-Data-2016/i94_oct16_sub.sas7bdat',
            './data/18-83510-I94-Data-2016/i94_nov16_sub.sas7bdat',
            './data/18-83510-I94-Data-2016/i94_dec16_sub.sas7bdat']

In [216]:
CustomI94Schema = R([
    Fld("cicid", Int()),
    Fld("i94yr", Int()),
    Fld("i94mon", Int()),  
    Fld("i94cit", Int()),  
    Fld("i94res", Int()),  
    Fld("i94port", Str()), 
    Fld("arrdate", Int()), 
    Fld("i94mode", Int()), 
    Fld("i94addr", Str()), 
    Fld("depdate", Int()), 
    Fld("i94bir", Int()),  
    Fld("i94visa", Int()), 
    Fld("count", Int()),
    Fld("dtadfile", Str()),
    Fld("visapost", Str()),
    Fld("occup", Str()),
    Fld("entdepa", Str()), 
    Fld("entdepd", Str()), 
    Fld("entdepu", Str()),
    Fld("matflag", Str()),  
    Fld("biryear", Int()), 
    Fld("dtaddto", Str()),
    Fld("gender", Str()),  
    Fld("insnum", Str()),  
    Fld("airline", Str()), 
    Fld("admnum", Dbl()),  
    Fld("fltno", Str()),
    Fld("visatype", Str())
])

CustomI94JunSchema = R([
    Fld("cicid", Int()),
    Fld("i94yr", Int()),
    Fld("i94mon", Int()),  
    Fld("i94cit", Int()),  
    Fld("i94res", Int()),  
    Fld("i94port", Str()), 
    Fld("arrdate", Int()), 
    Fld("i94mode", Int()), 
    Fld("i94addr", Str()), 
    Fld("depdate", Int()), 
    Fld("i94bir", Int()),  
    Fld("i94visa", Int()), 
    Fld("count", Int()),
    Fld("validres", Int()),
    Fld("delete_days", Int()),
    Fld("delete_mexl", Int()),
    Fld("delete_dup", Int()),
    Fld("delete_visa", Int()), 
    Fld("delete_recdup", Int()),
    Fld("dtadfile", Str()),
    Fld("visapost", Str()),
    Fld("occup", Str()),
    Fld("entdepa", Str()), 
    Fld("entdepd", Str()), 
    Fld("entdepu", Str()),
    Fld("matflag", Str()),  
    Fld("biryear", Int()), 
    Fld("dtaddto", Str()),
    Fld("gender", Str()),  
    Fld("insnum", Str()),  
    Fld("airline", Str()), 
    Fld("admnum", Dbl()),  
    Fld("fltno", Str()),
    Fld("visatype", Str())
])

In [224]:
# Examine i94 data
columns_to_drop = ['validres', 'delete_days', 'delete_mexl', 'delete_dup', 'delete_visa', 'delete_recdup']
df_i94s = OrderedDict() 
for f in i94_fnames:
    fname = os.path.basename(f)
    print(fname)
    if "jun" in fname: 
        d = spark.read.schema(CustomI94JunSchema).format('com.github.saurfang.sas.spark').load(f)
        d = d.drop(*columns_to_drop)
    else:
        d = spark.read.schema(CustomI94Schema).format('com.github.saurfang.sas.spark').load(f)
#     d = d.dropna(subset=["cicid", "arrdate", "i94addr"]).dropDuplicates()
    df_i94s[fname] = d
    d.printSchema()
    d.show()

i94_jan16_sub.sas7bdat
root
 |-- cicid: integer (nullable = true)
 |-- i94yr: integer (nullable = true)
 |-- i94mon: integer (nullable = true)
 |-- i94cit: integer (nullable = true)
 |-- i94res: integer (nullable = true)
 |-- i94port: string (nullable = true)
 |-- arrdate: integer (nullable = true)
 |-- i94mode: integer (nullable = true)
 |-- i94addr: string (nullable = true)
 |-- depdate: integer (nullable = true)
 |-- i94bir: integer (nullable = true)
 |-- i94visa: integer (nullable = true)
 |-- count: integer (nullable = true)
 |-- dtadfile: string (nullable = true)
 |-- visapost: string (nullable = true)
 |-- occup: string (nullable = true)
 |-- entdepa: string (nullable = true)
 |-- entdepd: string (nullable = true)
 |-- entdepu: string (nullable = true)
 |-- matflag: string (nullable = true)
 |-- biryear: integer (nullable = true)
 |-- dtaddto: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- insnum: string (nullable = true)
 |-- airline: string (nullable = tru

In [225]:
# define sample rate so that each month got same # of rows in union i94 df.
## get file size of each month i94 by ../../data/18-83510-I94-Data-2016/
# i94_filesizes = [415, 374, 459, 451, 501, 684, 620, 597, 543, 531, 424, 500]
# i94_samplerate = []
# N = 5
# for i in i94_filesizes:
#     i94_samplerate.append(N/i)
# print(i94_samplerate)

# i94_samplerate_dict = {}
# i = 0
# for k in df_i94s.keys():
#     i94_samplerate_dict[k] = i94_samplerate[i]
#     i = i+1
# pprint(i94_samplerate_dict)

In [226]:
# sample 1% of each month of i94, since origin size of sas is around 500MB.
sample_rate = 0.1
df_i94s_sample = {}
for k, v in df_i94s.items():
    df_i94s_sample[k] = v.sample(sample_rate)
    
from functools import reduce
print(len(df_i94s_sample.values()))
df_i94s_sample_unionall = reduce(DataFrame.unionAll, df_i94s_sample.values())

df_i94s_sample_unionall.printSchema()

12
root
 |-- cicid: integer (nullable = true)
 |-- i94yr: integer (nullable = true)
 |-- i94mon: integer (nullable = true)
 |-- i94cit: integer (nullable = true)
 |-- i94res: integer (nullable = true)
 |-- i94port: string (nullable = true)
 |-- arrdate: integer (nullable = true)
 |-- i94mode: integer (nullable = true)
 |-- i94addr: string (nullable = true)
 |-- depdate: integer (nullable = true)
 |-- i94bir: integer (nullable = true)
 |-- i94visa: integer (nullable = true)
 |-- count: integer (nullable = true)
 |-- dtadfile: string (nullable = true)
 |-- visapost: string (nullable = true)
 |-- occup: string (nullable = true)
 |-- entdepa: string (nullable = true)
 |-- entdepd: string (nullable = true)
 |-- entdepu: string (nullable = true)
 |-- matflag: string (nullable = true)
 |-- biryear: integer (nullable = true)
 |-- dtaddto: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- insnum: string (nullable = true)
 |-- airline: string (nullable = true)
 |-- admnum: doub

In [227]:
df_i94s_sample_unionall.write.mode('overwrite').parquet(path='i94_sample.parquet')

In [228]:
%%time
df_sas = spark.read.parquet('sas_data/')
df_sas.count()

CPU times: user 3.53 ms, sys: 99 µs, total: 3.63 ms
Wall time: 281 ms


3096313

In [270]:
%%time
df_imm = spark.read.parquet('i94_sample.parquet/')
df_imm = df_imm.dropna(subset=["cicid", "arrdate", "i94addr"]).dropDuplicates()
df_imm = df_imm.filter( df_imm.i94cit.isin(list(meta_i94['I94CIT'].keys())) )
df_imm = df_imm.filter( df_imm.i94res.isin(list(meta_i94['I94CIT'].keys())) )
LOG.info(df_imm.count())
df_imm.printSchema()
df_imm.show()

2020-09-30 16:34:10,697 INFO  [etl] 3402211


root
 |-- cicid: integer (nullable = true)
 |-- i94yr: integer (nullable = true)
 |-- i94mon: integer (nullable = true)
 |-- i94cit: integer (nullable = true)
 |-- i94res: integer (nullable = true)
 |-- i94port: string (nullable = true)
 |-- arrdate: integer (nullable = true)
 |-- i94mode: integer (nullable = true)
 |-- i94addr: string (nullable = true)
 |-- depdate: integer (nullable = true)
 |-- i94bir: integer (nullable = true)
 |-- i94visa: integer (nullable = true)
 |-- count: integer (nullable = true)
 |-- dtadfile: string (nullable = true)
 |-- visapost: string (nullable = true)
 |-- occup: string (nullable = true)
 |-- entdepa: string (nullable = true)
 |-- entdepd: string (nullable = true)
 |-- entdepu: string (nullable = true)
 |-- matflag: string (nullable = true)
 |-- biryear: integer (nullable = true)
 |-- dtaddto: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- insnum: string (nullable = true)
 |-- airline: string (nullable = true)
 |-- admnum: double 

In [416]:
# extract valid i94 data
drop_columns = ['i94mode', 'i94bir', 'count', 'dtadfile', 'visapost', 'occup', 'entdepa', 'entdepd', 'entdepu', 'matflag', 'dtaddto', 'insnum', 'admnum', 'fltno']
df_imm = df_imm.drop(*drop_columns)
df_imm.printSchema()
df_imm.show()

root
 |-- cicid: integer (nullable = true)
 |-- i94yr: integer (nullable = true)
 |-- i94mon: integer (nullable = true)
 |-- i94cit: integer (nullable = true)
 |-- i94res: integer (nullable = true)
 |-- i94port: string (nullable = true)
 |-- arrdate: integer (nullable = true)
 |-- i94addr: string (nullable = true)
 |-- depdate: integer (nullable = true)
 |-- i94visa: integer (nullable = true)
 |-- biryear: integer (nullable = true)
 |-- gender: string (nullable = true)
 |-- airline: string (nullable = true)
 |-- visatype: string (nullable = true)

+-----+-----+------+------+------+-------+-------+-------+-------+-------+-------+------+-------+--------+
|cicid|i94yr|i94mon|i94cit|i94res|i94port|arrdate|i94addr|depdate|i94visa|biryear|gender|airline|visatype|
+-----+-----+------+------+------+-------+-------+-------+-------+-------+-------+------+-------+--------+
| 2187| 2016|    12|   209|   209|    AGA|  20789|     FL|   null|      2|   1957|     M|     UA|     GMT|
| 5708| 2016|    1

In [418]:
# write valid i94 into parquet for Spark
df_imm.write.mode('overwrite').parquet(path='i94_valid.parquet')

In [419]:
staging_imm = df_imm.createOrReplaceTempView("imm")

In [420]:
imm_table = spark.sql("""
select distinct i94mon, COUNT(*)
FROM imm
GROUP BY 1
ORDER BY 1 asc
""")
imm_table.printSchema()
imm_table.show()
LOG.info(f"total 1% sample parquet rows: {imm_table.count()}")

root
 |-- i94mon: integer (nullable = true)
 |-- count(1): long (nullable = false)

+------+--------+
|i94mon|count(1)|
+------+--------+
|     1|  229728|
|     2|  207544|
|     3|  258221|
|     4|  258500|
|     5|  284546|
|     6|  298284|
|     7|  362330|
|     8|  349680|
|     9|  311740|
|    10|  305532|
|    11|  243670|
|    12|  292436|
+------+--------+



2020-09-30 19:51:44,468 INFO  [etl] total 1% sample parquet rows: 12


In [232]:
test = spark.sql("""
select cicid, count(*)
from imm
group by 1
having count(cicid) > 1
order by 2
""")
test.printSchema()
test.show()
test.count()

root
 |-- cicid: integer (nullable = true)
 |-- count(1): long (nullable = false)

+-------+--------+
|  cicid|count(1)|
+-------+--------+
|4643758|       2|
|5133024|       2|
|4758513|       2|
|6106423|       2|
| 446447|       2|
|1735689|       2|
| 454087|       2|
|3797978|       2|
| 914792|       2|
|1991821|       2|
| 373035|       2|
|3688657|       2|
|2535632|       2|
|4329643|       2|
|3066636|       2|
| 806482|       2|
|4334036|       2|
|1805074|       2|
|2095808|       2|
|3960505|       2|
+-------+--------+
only showing top 20 rows



722568

## demographics data

In [372]:
custom_demo_schema = R([
    Fld("City", Str()),
    Fld("State", Str()),
    Fld("Median_Age", Dbl()),  
    Fld("Male_Population", Int()),
    Fld("Female_Population", Int()),
    Fld("Total_Population", Int()),
    Fld("Number_Veterans", Int()),
    Fld("Foregin_born", Int()),
    Fld("Average_Household", Dbl()),
    Fld("State_Code", Str()),
    Fld("Race", Str()),
    Fld("Count", Int()),
])

df_degh = spark.read.schema(custom_demo_schema).options(header='True',sep=';').csv('./us-cities-demographics.csv')
df_degh.printSchema()
df_degh.show(5)

root
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Median_Age: double (nullable = true)
 |-- Male_Population: integer (nullable = true)
 |-- Female_Population: integer (nullable = true)
 |-- Total_Population: integer (nullable = true)
 |-- Number_Veterans: integer (nullable = true)
 |-- Foregin_born: integer (nullable = true)
 |-- Average_Household: double (nullable = true)
 |-- State_Code: string (nullable = true)
 |-- Race: string (nullable = true)
 |-- Count: integer (nullable = true)

+----------------+-------------+----------+---------------+-----------------+----------------+---------------+------------+-----------------+----------+--------------------+-----+
|            City|        State|Median_Age|Male_Population|Female_Population|Total_Population|Number_Veterans|Foregin_born|Average_Household|State_Code|                Race|Count|
+----------------+-------------+----------+---------------+-----------------+----------------+---------------+----

## Checkout airport data

In [261]:
df_air = spark.read.options(header='True',inferSchema='True').csv('./airport-codes_csv.csv')
df_air = df_air.dropna(subset=["iata_code"])
df_air.printSchema()
df_air.show(5)
df_air.count()

root
 |-- ident: string (nullable = true)
 |-- type: string (nullable = true)
 |-- name: string (nullable = true)
 |-- elevation_ft: integer (nullable = true)
 |-- continent: string (nullable = true)
 |-- iso_country: string (nullable = true)
 |-- iso_region: string (nullable = true)
 |-- municipality: string (nullable = true)
 |-- gps_code: string (nullable = true)
 |-- iata_code: string (nullable = true)
 |-- local_code: string (nullable = true)
 |-- coordinates: string (nullable = true)

+-----+-------------+--------------------+------------+---------+-----------+----------+-------------+--------+---------+----------+--------------------+
|ident|         type|                name|elevation_ft|continent|iso_country|iso_region| municipality|gps_code|iata_code|local_code|         coordinates|
+-----+-------------+--------------------+------------+---------+-----------+----------+-------------+--------+---------+----------+--------------------+
|  03N|small_airport|      Utirik Airport|

9189

In [421]:
df_air.write.mode('overwrite').parquet(path='df_airport.parquet')

## Checkout temperature data

In [236]:
df_tmpt = spark.read.options(header='True', inferSchema='True').csv('./GlobalLandTemperaturesByCity.csv')
df_tmpt = df_tmpt.filter(df_tmpt.Country == "United States")
df_tmpt.printSchema()
df_tmpt.show(5)

root
 |-- dt: string (nullable = true)
 |-- AverageTemperature: double (nullable = true)
 |-- AverageTemperatureUncertainty: double (nullable = true)
 |-- City: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Latitude: string (nullable = true)
 |-- Longitude: string (nullable = true)

+----------+------------------+-----------------------------+-------+-------------+--------+---------+
|        dt|AverageTemperature|AverageTemperatureUncertainty|   City|      Country|Latitude|Longitude|
+----------+------------------+-----------------------------+-------+-------------+--------+---------+
|1820-01-01|2.1010000000000004|                        3.217|Abilene|United States|  32.95N|  100.53W|
|1820-02-01|             6.926|                        2.853|Abilene|United States|  32.95N|  100.53W|
|1820-03-01|            10.767|                        2.395|Abilene|United States|  32.95N|  100.53W|
|1820-04-01|17.988999999999994|                        2.202|Abilene|United

In [237]:
df_tmpt.write.mode('overwrite').parquet(path='temperature_us.parquet')

In [258]:
df_temper = spark.read.parquet('./temperature_us.parquet/')
df_temper = df_temper.withColumn("datetime", F.col("dt").cast("date")).drop("dt")
df_temper = df_temper.filter( df_temper["datetime"] >= F.lit('2016-01-01') )
df_temper.printSchema()
df_temper.show()

root
 |-- AverageTemperature: double (nullable = true)
 |-- AverageTemperatureUncertainty: double (nullable = true)
 |-- City: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Latitude: string (nullable = true)
 |-- Longitude: string (nullable = true)
 |-- datetime: date (nullable = true)

+------------------+-----------------------------+----+-------+--------+---------+--------+
|AverageTemperature|AverageTemperatureUncertainty|City|Country|Latitude|Longitude|datetime|
+------------------+-----------------------------+----+-------+--------+---------+--------+
+------------------+-----------------------------+----+-------+--------+---------+--------+



### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
Map out the conceptual data model and explain why you chose that model

#### 3.2 Mapping Out Data Pipelines
List the steps necessary to pipeline the data into the chosen data model

## Extract i94 table from dataframe

In [311]:
%%time
df_i94 = spark.read.parquet('i94_valid.parquet/')
df_i94 = df_i94.filter( df_i94.i94cit.isin(set(meta_i94['I94CIT'].keys())) )
df_i94 = df_i94.filter( df_i94.i94res.isin(set(meta_i94['I94CIT'].keys())) )
df_i94.show()

+-----+-----+------+------+------+-------+-------+-------+-------+-------+------+-------+-----+--------+--------+-----+-------+-------+-------+-------+-------+--------+------+------+-------+---------------+-----+--------+
|cicid|i94yr|i94mon|i94cit|i94res|i94port|arrdate|i94mode|i94addr|depdate|i94bir|i94visa|count|dtadfile|visapost|occup|entdepa|entdepd|entdepu|matflag|biryear| dtaddto|gender|insnum|airline|         admnum|fltno|visatype|
+-----+-----+------+------+------+-------+-------+-------+-------+-------+------+-------+-----+--------+--------+-----+-------+-------+-------+-------+-------+--------+------+------+-------+---------------+-----+--------+
| 7228| 2016|    12|   209|   209|    AGA|  20789|      1|     NY|  20793|    35|      2|    1|20161201|    null| null|      A|      D|   null|      M|   1981|01142017|     M|  3680|     JL|4.0247459128E10|00941|     GMT|
| 8754| 2016|     8|   582|   582|    SNA|  20667|      1|     CA|  20674|    23|      2|    1|20160808|    null

# Create i94 staging table

In [349]:
df_i94.createOrReplaceTempView("i94")
i94_stage = spark.sql("""
SELECT 
    distinct cicid as cid,
    i94yr,
    i94mon,
    EXTRACT(DAY FROM date_add('1960-01-01',arrdate) ) as i94day,
    i94cit,
    i94res,
    i94port,
    i94addr,
    i94visa,
    biryear,
    gender,
    airline,
    visatype,
    date_add('1960-01-01',arrdate) as arrdate,
    date_add('1960-01-01',depdate) as depdate
FROM i94
order by i94cit asc
""")
i94_stage.printSchema()
i94_stage.show()

root
 |-- cid: integer (nullable = true)
 |-- i94yr: integer (nullable = true)
 |-- i94mon: integer (nullable = true)
 |-- i94day: integer (nullable = true)
 |-- i94cit: integer (nullable = true)
 |-- i94res: integer (nullable = true)
 |-- i94port: string (nullable = true)
 |-- i94addr: string (nullable = true)
 |-- i94visa: integer (nullable = true)
 |-- biryear: integer (nullable = true)
 |-- gender: string (nullable = true)
 |-- airline: string (nullable = true)
 |-- visatype: string (nullable = true)
 |-- arrdate: date (nullable = true)
 |-- depdate: date (nullable = true)

+-------+-----+------+------+------+------+-------+-------+-------+-------+------+-------+--------+----------+----------+
|    cid|i94yr|i94mon|i94day|i94cit|i94res|i94port|i94addr|i94visa|biryear|gender|airline|visatype|   arrdate|   depdate|
+-------+-----+------+------+------+------+-------+-------+-------+-------+------+-------+--------+----------+----------+
|1003688| 2016|     5|     6|   101|   101|    MI

# Create airport dim table

In [344]:
meta_i94.keys()

dict_keys(['I94CIT', 'I94PORT', 'I94MODE', 'I94ADDR'])

In [404]:
i94port = meta_i94['I94PORT']
pddf = pd.DataFrame(i94port.items(), columns=["id", "airport"])
df_i94port = spark.createDataFrame(pddf)
df_i94port.printSchema()
df_i94port.show()

root
 |-- id: string (nullable = true)
 |-- airport: string (nullable = true)

+---+--------------------+
| id|             airport|
+---+--------------------+
|ALC|           ALCAN, AK|
|ANC|       ANCHORAGE, AK|
|BAR|BAKER AAF - BAKER...|
|DAC|   DALTONS CACHE, AK|
|PIZ|DEW STATION PT LA...|
|DTH|    DUTCH HARBOR, AK|
|EGL|           EAGLE, AK|
|FRB|       FAIRBANKS, AK|
|HOM|           HOMER, AK|
|HYD|           HYDER, AK|
|JUN|          JUNEAU, AK|
|5KE|       KETCHIKAN, AK|
|KET|       KETCHIKAN, AK|
|MOS|MOSES POINT INTER...|
|NIK|         NIKISKI, AK|
|NOM|             NOM, AK|
|PKC|     POKER CREEK, AK|
|ORI|  PORT LIONS SPB, AK|
|SKA|         SKAGWAY, AK|
|SNP| ST. PAUL ISLAND, AK|
+---+--------------------+
only showing top 20 rows



In [405]:
df_i94port.write.mode('overwrite').parquet(path='df_i94port.parquet')

# Create Visa table for 1,2,3

In [429]:
visa = meta_i94['I94VISA']
print(visa)

{1: 'BUSINESS', 2: 'PLEASURE', 3: 'STUDENT'}


In [430]:
pddf = pd.DataFrame(visa.items(), columns=["id", "visa_type"])
df_i94visa = spark.createDataFrame(pddf)
df_i94visa.printSchema()
df_i94visa.show()

root
 |-- id: long (nullable = true)
 |-- visa_type: string (nullable = true)

+---+---------+
| id|visa_type|
+---+---------+
|  1| BUSINESS|
|  2| PLEASURE|
|  3|  STUDENT|
+---+---------+



In [431]:
df_i94visa.write.mode('overwrite').parquet(path='df_i94visa.parquet')

# Create cit_res dim table

In [406]:
cit = meta_i94['I94CIT']
pddf = pd.DataFrame(cit.items(), columns=["id", "citres"])
df_i94cit = spark.createDataFrame(pddf)
df_i94cit.printSchema()
df_i94cit.show()

root
 |-- id: long (nullable = true)
 |-- citres: string (nullable = true)

+---+--------------------+
| id|              citres|
+---+--------------------+
|582|MEXICO AIR SEA, A...|
|236|         AFGHANISTAN|
|101|             ALBANIA|
|316|             ALGERIA|
|102|             ANDORRA|
|324|              ANGOLA|
|529|            ANGUILLA|
|518|     ANTIGUA-BARBUDA|
|687|           ARGENTINA|
|151|             ARMENIA|
|532|               ARUBA|
|438|           AUSTRALIA|
|103|             AUSTRIA|
|152|          AZERBAIJAN|
|512|             BAHAMAS|
|298|             BAHRAIN|
|274|          BANGLADESH|
|513|            BARBADOS|
|104|             BELGIUM|
|581|              BELIZE|
+---+--------------------+
only showing top 20 rows



In [407]:
df_i94cit.write.mode('overwrite').parquet(path='df_i94cit.parquet')

# Create addr dim table

In [408]:
meta_i94.keys()
addr = meta_i94['I94ADDR']
pddf = pd.DataFrame(addr.items(), columns=["st", "state"])
df_i94addr = spark.createDataFrame(pddf)
df_i94addr.printSchema()
df_i94addr.show()

da_1 = df_i94addr.filter( df_i94addr.st == 'CA')
da_1.show()

root
 |-- st: string (nullable = true)
 |-- state: string (nullable = true)

+---+-----------------+
| st|            state|
+---+-----------------+
| AL|          ALABAMA|
| AK|           ALASKA|
| AZ|          ARIZONA|
| AR|         ARKANSAS|
| CA|       CALIFORNIA|
| CO|         COLORADO|
| CT|      CONNECTICUT|
| DE|         DELAWARE|
| DC|DIST. OF COLUMBIA|
| FL|          FLORIDA|
| GA|          GEORGIA|
| GU|             GUAM|
| HI|           HAWAII|
| ID|            IDAHO|
| IL|         ILLINOIS|
| IN|          INDIANA|
| IA|             IOWA|
| KS|           KANSAS|
| KY|         KENTUCKY|
| LA|        LOUISIANA|
+---+-----------------+
only showing top 20 rows

+---+----------+
| st|     state|
+---+----------+
| CA|CALIFORNIA|
+---+----------+



In [409]:
df_i94addr.write.mode('overwrite').parquet(path='df_i94addr.parquet')

# Create demopgrahics dim table

In [395]:
# df_demo_1 = df_degh.filter(df_degh["State_Code"] == "ID")
df_demo_1 = df_degh
df_demo_1.printSchema()
df_demo_1.show()

root
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Median_Age: double (nullable = true)
 |-- Male_Population: integer (nullable = true)
 |-- Female_Population: integer (nullable = true)
 |-- Total_Population: integer (nullable = true)
 |-- Number_Veterans: integer (nullable = true)
 |-- Foregin_born: integer (nullable = true)
 |-- Average_Household: double (nullable = true)
 |-- State_Code: string (nullable = true)
 |-- Race: string (nullable = true)
 |-- Count: integer (nullable = true)

+----------------+--------------+----------+---------------+-----------------+----------------+---------------+------------+-----------------+----------+--------------------+------+
|            City|         State|Median_Age|Male_Population|Female_Population|Total_Population|Number_Veterans|Foregin_born|Average_Household|State_Code|                Race| Count|
+----------------+--------------+----------+---------------+-----------------+----------------+---------------

In [396]:
# https://databricks.com/blog/2016/02/09/reshaping-data-with-pivot-in-apache-spark.html
df_pi = df_demo_1.groupby("State_Code","Median_Age","Male_Population","Female_Population","Total_Population","Number_Veterans","Foregin_born","Average_Household").pivot("Race").sum("Count")
df_pi.printSchema()
df_pi.show()

root
 |-- State_Code: string (nullable = true)
 |-- Median_Age: double (nullable = true)
 |-- Male_Population: integer (nullable = true)
 |-- Female_Population: integer (nullable = true)
 |-- Total_Population: integer (nullable = true)
 |-- Number_Veterans: integer (nullable = true)
 |-- Foregin_born: integer (nullable = true)
 |-- Average_Household: double (nullable = true)
 |-- American Indian and Alaska Native: long (nullable = true)
 |-- Asian: long (nullable = true)
 |-- Black or African-American: long (nullable = true)
 |-- Hispanic or Latino: long (nullable = true)
 |-- White: long (nullable = true)

+----------+----------+---------------+-----------------+----------------+---------------+------------+-----------------+---------------------------------+-----+-------------------------+------------------+------+
|State_Code|Median_Age|Male_Population|Female_Population|Total_Population|Number_Veterans|Foregin_born|Average_Household|American Indian and Alaska Native|Asian|Black or A

In [411]:
df_demo = df_pi.groupby('State_Code').agg(F.avg('Median_Age').alias('Median_Age'), \
                                         F.sum('Male_Population').alias('Male_Population'), \
                                         F.sum('Female_Population').alias('Female_Population'), \
                                         F.sum('Total_Population').alias('Total_Population'), \
                                         F.sum('Number_Veterans').alias('Number_Veterans'), 
                                         F.sum('Foregin_born').alias('Foregin_born'), \
                                         F.avg('Average_Household').alias('Average_Household'), \
                                         F.sum('American Indian and Alaska Native').alias('American_Indian_Alaska_Native'), \
                                         F.sum('Asian').alias('Asian'), \
                                         F.sum('Black or African-American').alias('Black_African-American'), \
                                         F.sum('Hispanic or Latino').alias('Hispanic_Latino'), \
                                         F.sum('White').alias('White') \
                                        )
df_demo.printSchema()
df_demo.show()

root
 |-- State_Code: string (nullable = true)
 |-- Median_Age: double (nullable = true)
 |-- Male_Population: long (nullable = true)
 |-- Female_Population: long (nullable = true)
 |-- Total_Population: long (nullable = true)
 |-- Number_Veterans: long (nullable = true)
 |-- Foregin_born: long (nullable = true)
 |-- Average_Household: double (nullable = true)
 |-- American_Indian_Alaska_Native: long (nullable = true)
 |-- Asian: long (nullable = true)
 |-- Black_African-American: long (nullable = true)
 |-- Hispanic_Latino: long (nullable = true)
 |-- White: long (nullable = true)

+----------+------------------+---------------+-----------------+----------------+---------------+------------+------------------+-----------------------------+-------+----------------------+---------------+--------+
|State_Code|        Median_Age|Male_Population|Female_Population|Total_Population|Number_Veterans|Foregin_born| Average_Household|American_Indian_Alaska_Native|  Asian|Black_African-American|Hi

In [412]:
df_demo.write.mode('overwrite').parquet(path='df_demo.parquet')

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.

# Build fact table

#### 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
 * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
 * Unit tests for the scripts to ensure they are doing the right thing
 * Source/Count checks to ensure completeness
 
Run Quality Checks

In [ ]:
# Perform quality checks here

#### 4.3 Data dictionary 
Create a data dictionary for your data model. For each field, provide a brief description of what the data is and where it came from. You can include the data dictionary in the notebook or in a separate file.

#### Step 5: Complete Project Write Up
* Clearly state the rationale for the choice of tools and technologies for the project.
* Propose how often the data should be updated and why.
* Write a description of how you would approach the problem differently under the following scenarios:
 * The data was increased by 100x.
 * The data populates a dashboard that must be updated on a daily basis by 7am every day.
 * The database needed to be accessed by 100+ people.

# Expriment here

In [160]:
print(sections_dict['I94CIT'])

{'582': 'MEXICO AIR SEA, AND NOT REPORTED (I-94, NO LAND ARRIVALS)', '236': 'AFGHANISTAN', '101': 'ALBANIA', '316': 'ALGERIA', '102': 'ANDORRA', '324': 'ANGOLA', '529': 'ANGUILLA', '518': 'ANTIGUA-BARBUDA', '687': 'ARGENTINA', '151': 'ARMENIA', '532': 'ARUBA', '438': 'AUSTRALIA', '103': 'AUSTRIA', '152': 'AZERBAIJAN', '512': 'BAHAMAS', '298': 'BAHRAIN', '274': 'BANGLADESH', '513': 'BARBADOS', '104': 'BELGIUM', '581': 'BELIZE', '386': 'BENIN', '509': 'BERMUDA', '153': 'BELARUS', '242': 'BHUTAN', '688': 'BOLIVIA', '717': 'BONAIRE, ST EUSTATIUS, SABA', '164': 'BOSNIA-HERZEGOVINA', '336': 'BOTSWANA', '689': 'BRAZIL', '525': 'BRITISH VIRGIN ISLANDS', '217': 'BRUNEI', '105': 'BULGARIA', '393': 'BURKINA FASO', '243': 'BURMA', '375': 'BURUNDI', '310': 'CAMEROON', '326': 'CAPE VERDE', '526': 'CAYMAN ISLANDS', '383': 'CENTRAL AFRICAN REPUBLIC', '384': 'CHAD', '690': 'CHILE', '245': 'CHINA, PRC', '721': 'CURACAO', '270': 'CHRISTMAS ISLAND', '271': 'COCOS ISLANDS', '691': 'COLOMBIA', '317': 'COMOR

In [161]:
print(sections_dict['I94MODE'])

{'1': 'AIR', '2': 'SEA', '3': 'LAND'}


In [162]:
print(meta_i94['I94MODE'])

{1: 'AIR', 2: 'SEA', 3: 'LAND'}


In [163]:
print(meta_i94['I94PORT'])

{'ALC': 'ALCAN, AK', 'ANC': 'ANCHORAGE, AK', 'BAR': 'BAKER AAF - BAKER ISLAND, AK', 'DAC': 'DALTONS CACHE, AK', 'PIZ': 'DEW STATION PT LAY DEW, AK', 'DTH': 'DUTCH HARBOR, AK', 'EGL': 'EAGLE, AK', 'FRB': 'FAIRBANKS, AK', 'HOM': 'HOMER, AK', 'HYD': 'HYDER, AK', 'JUN': 'JUNEAU, AK', '5KE': 'KETCHIKAN, AK', 'KET': 'KETCHIKAN, AK', 'MOS': 'MOSES POINT INTERMEDIATE, AK', 'NIK': 'NIKISKI, AK', 'NOM': 'NOM, AK', 'PKC': 'POKER CREEK, AK', 'ORI': 'PORT LIONS SPB, AK', 'SKA': 'SKAGWAY, AK', 'SNP': 'ST. PAUL ISLAND, AK', 'TKI': 'TOKEEN, AK', 'WRA': 'WRANGELL, AK', 'HSV': 'MADISON COUNTY - HUNTSVILLE, AL', 'MOB': 'MOBILE, AL', 'LIA': 'LITTLE ROCK, AR (BPS)', 'ROG': 'ROGERS ARPT, AR', 'DOU': 'DOUGLAS, AZ', 'LUK': 'LUKEVILLE, AZ', 'MAP': 'MARIPOSA AZ', 'NAC': 'NACO, AZ', 'NOG': 'NOGALES, AZ', 'PHO': 'PHOENIX, AZ', 'POR': 'PORTAL, AZ', 'SLU': 'SAN LUIS, AZ', 'SAS': 'SASABE, AZ', 'TUC': 'TUCSON, AZ', 'YUI': 'YUMA, AZ', 'AND': 'ANDRADE, CA', 'BUR': 'BURBANK, CA', 'CAL': 'CALEXICO, CA', 'CAO': 'CAMPO, CA

In [303]:
print(list(meta_i94['I94CIT'].keys()) )

[582, 236, 101, 316, 102, 324, 529, 518, 687, 151, 532, 438, 103, 152, 512, 298, 274, 513, 104, 581, 386, 509, 153, 242, 688, 717, 164, 336, 689, 525, 217, 105, 393, 243, 375, 310, 326, 526, 383, 384, 690, 245, 721, 270, 271, 691, 317, 385, 467, 575, 165, 584, 218, 140, 723, 108, 322, 519, 585, 240, 692, 368, 576, 399, 372, 109, 369, 604, 413, 110, 111, 601, 411, 387, 338, 758, 154, 112, 339, 143, 113, 520, 507, 577, 382, 327, 603, 586, 726, 149, 528, 206, 114, 115, 213, 759, 729, 204, 249, 250, 116, 251, 117, 388, 514, 209, 253, 201, 155, 340, 414, 732, 272, 156, 203, 118, 255, 335, 370, 381, 119, 120, 121, 214, 167, 320, 345, 273, 220, 392, 145, 472, 511, 389, 342, 760, 473, 157, 122, 299, 735, 521, 332, 329, 371, 440, 257, 123, 508, 409, 464, 579, 390, 343, 470, 275, 124, 256, 258, 474, 743, 504, 441, 693, 694, 260, 416, 107, 126, 297, 748, 321, 127, 158, 376, 128, 330, 261, 391, 142, 745, 347, 348, 207, 141, 166, 412, 397, 373, 276, 129, 244, 346, 522, 523, 502, 524, 716, 736, 749,

In [ ]:
## fix SAS before save to parquet